**MIT License**

Copyright (c) [2019] [Chinh Ngo]

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Install dependencies.

In [1]:
# Imports we need.
%tensorflow_version 1.x

import tensorflow as tf

# Install Tensor2tensor
!pip install -q -U tensor2tensor
!pip install tensorflow-datasets==3.2.1

print('All done.')


TensorFlow 1.x selected.
All done.


# Setup some options.

In [2]:
import numpy as np
import os
from os import path
import collections
import json
import pprint

problem = 'fb_wiki_and_book_m45_translate_envi_iwslt32k'
hparams_set = 'transformer_tall9'

google_cloud_bucket = 'ntkchinh_public'
data_dir = 'translation_envi'  
logdir = 'translation_envi' 
tmp_dir = 'translation_envi'

In [3]:
# Check if the runtime is set to TPU or GPU:
use_tpu = True
# use_tpu = False

  
def setup_tpu():
  from google.colab import auth
  auth.authenticate_user()

  # Mount the bucket to colab
  # First we install gcsfuse to be able to mount Google Cloud Storage with Colab.
  print('\nInstalling gcsfuse')
  !echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
  !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
  !apt -qq update
  !apt -qq install gcsfuse

  bucket = google_cloud_bucket
  print('Mounting bucket {} to local.'.format(bucket))
  mount_point = '/content/{}'.format(bucket)
  if not os.path.exists(mount_point):
    tf.gfile.MakeDirs(mount_point)
  
  !fusermount -u $mount_point
  !gcsfuse --implicit-dirs $bucket $mount_point
  print('\nMount point content:')
  !ls $mount_point

  # First we Connect to the TPU pod.
  tpu_address = ''
  if 'COLAB_TPU_ADDR' in os.environ:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)
    with tf.Session(tpu_address) as session:
      devices = session.list_devices()
      # Upload credentials to TPU.
      with open('/content/adc.json', 'r') as f:
        auth_info = json.load(f)
      tf.contrib.cloud.configure_gcs(session, credentials=auth_info)

    print('TPU devices:')
    pprint.pprint(devices)

  return mount_point, tpu_address


if not use_tpu:
  mount_point = setup_gpu()
  tpu_address = ''
else:
  mount_point, tpu_address = setup_tpu()
  
print('\nMount point: {}'.format(mount_point))
print('TPU address: {}'.format(tpu_address))


Installing gcsfuse
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1974  100  1974    0     0  75923      0 --:--:-- --:--:-- --:--:-- 75923
OK
20 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.33.1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Mounting bucket ntkchinh_public to local.
Using mount point: /content/ntkchinh_public
Opening GCS connection...
Mounting file system...
File system has been successfully mounted.

Mount point content:
translation_envi  translation_vien
TPU address is grpc://10.25.25.42:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12160641460738676782),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4759817261275541018),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device

In [4]:
# Now we make all the paths absolute.
logdir = os.path.join(mount_point, logdir)
data_dir = os.path.join(mount_point, data_dir)
tmp_dir = os.path.join(mount_point, tmp_dir)

run_logdir = logdir

print('log dir: {}'.format(run_logdir))
print('data dir: {}'.format(data_dir))
print('temp dir: {}'.format(tmp_dir))

log dir: /content/ntkchinh_public/translation_envi
data dir: /content/ntkchinh_public/translation_envi
temp dir: /content/ntkchinh_public/translation_envi


In [5]:
src = '/content/translation_vien'
if not os.path.exists(src):
  os.chdir('/content')
  ! git clone https://github.com/ntkchinh/translation_vien.git
else:
  % cd $src
  ! git pull
  % cd /

print('\n Source code:')
%ls $src

/content/translation_vien
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/ntkchinh/translation_vien
   4261e70..9f08f1f  master     -> origin/master
Updating 4261e70..9f08f1f
Fast-forward
 Public_access_to_Translation_Model.ipynb | 697 +++++++++++++++++++++++++++++++
 1 file changed, 697 insertions(+)
 create mode 100644 Public_access_to_Translation_Model.ipynb
/

 Source code:
problems.py                               __pycache__/
Public_access_to_Translation_Model.ipynb  t2t_decoder.py


#Decode & compute BLEU score

In [6]:
decode_from_file = os.path.join(tmp_dir, 'tst2013.en')
decode_to_file = 'tst2013.en2vi'  # no longer write to cloud bucket.
ref_file = os.path.join(tmp_dir, 'tst2013.vi')

if use_tpu:
  # TPU wants the paths to begin with gs://
  ckpt_dir = logdir.replace(mount_point, 'gs://{}'.format(google_cloud_bucket))
ckpt_path = os.path.join(ckpt_dir, 'model.ckpt')

print('Decode to file {}'.format(decode_to_file))
!python $src/t2t_decoder.py \
--data_dir=$data_dir --problem=$problem \
--hparams_set=$hparams_set \
--model=transformer \
--decode_hparams="beam_size=6,alpha=0.6"  \
--decode_from_file=$decode_from_file \
--decode_to_file=$decode_to_file  \
--checkpoint_path=$ckpt_path  \
--output_dir=$ckpt_dir  \
--use_tpu=$use_tpu \
--cloud_tpu_name=$tpu_address


Streaming output truncated to the last 5000 lines.
INFO:tensorflow:Inference results INPUT: There are the New World vultures that are mainly found in the Americas , like the condors and the caracaras , and then the Old World vultures , where we have 16 species . From these 16 , 11 of them are facing a high risk of extinction .
I0208 17:08:34.272921 140699990935424 decoding.py:155] Inference results INPUT: There are the New World vultures that are mainly found in the Americas , like the condors and the caracaras , and then the Old World vultures , where we have 16 species . From these 16 , 11 of them are facing a high risk of extinction .
INFO:tensorflow:Inference results OUTPUT: Có những loài kền kền Tân thế giới chủ yếu được tìm thấy ở châu Mỹ , giống như chim cốc và chim hoàng yến , và sau đó là kền Cựu thế giới , nơi chúng ta có 16 loài . Từ 16 . 11 loài này đang đối mặt với nguy cơ tuyệt chủng cao .
I0208 17:08:34.273854 140699990935424 decoding.py:170] Inference results OUTPUT: Có

In [7]:
!head -10 $decode_to_file

Khi còn nhỏ , tôi nghĩ đất nước mình là tốt nhất trên hành tinh này , và tôi lớn lên hát một bài hát tên là &quot; Không có gì để ghen tị . &quot;
Và tôi đã rất tự hào .
Ở trường , chúng tôi dành rất nhiều thời gian nghiên cứu lịch sử của Kim Il-Sung , nhưng chúng tôi chưa bao giờ biết nhiều về thế giới bên ngoài , ngoại trừ Mỹ , Hàn Quốc , Nhật Bản là kẻ thù .
Mặc dù tôi thường băn khoăn về thế giới bên ngoài , tôi nghĩ mình sẽ dành cả cuộc đời mình ở Bắc Triều Tiên , cho đến khi mọi thứ đột nhiên thay đổi .
Khi tôi 7 tuổi , tôi chứng kiến lần hành quyết trước công chúng đầu tiên , nhưng tôi nghĩ cuộc sống của tôi ở Bắc Triều Tiên là bình thường .
Gia đình tôi không nghèo , và bản thân tôi , tôi chưa bao giờ trải qua cơn đói .
Nhưng một ngày , vào năm 1995 , mẹ tôi mang về nhà một lá thư từ chị của đồng nghiệp
Nó viết , &quot; Khi bạn đọc cái này , cả năm thành viên trong gia đình sẽ không tồn tại trên thế giới này , bởi vì chúng tôi đã không ăn trong hai tuần qua .
Chúng ta cùng nằm 

In [8]:
print('\nCompare {} with reference {}'.format(decode_to_file, ref_file))
!t2t-bleu --translation=$decode_to_file --reference=$ref_file


Compare tst2013.en2vi with reference /content/ntkchinh_public/translation_envi/tst2013.vi



BLEU_uncased =  37.60
BLEU_cased =  36.72
